<a href="https://colab.research.google.com/github/Sayed-Ali-Raza-Naqvi/mlops-zoomcamp-homeworks/blob/main/Homework_Module1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import pandas as pd
import pyarrow
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [119]:
def read_data(file):
  df = pd.read_parquet(file)

  df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
  df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

  df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
  df['duration'] = df['duration'].apply(lambda x: x.total_seconds()/60)

  df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

  return df

In [120]:
df_train = read_data('/content/yellow_tripdata_2023-01.parquet')
df_val = read_data('/content/yellow_tripdata_2023-02.parquet')

In [122]:
data_to_encode = ['PULocationID', 'DOLocationID']
df_train[data_to_encode] = df_train[data_to_encode].astype(str)
df_val[data_to_encode] = df_val[data_to_encode].astype(str)

In [124]:
train_dict = df_train[data_to_encode].to_dict(orient='records')
val_dict = df_val[data_to_encode].to_dict(orient='records')

In [125]:
dict_vectorizer = DictVectorizer()

In [133]:
X_train = dict_vectorizer.fit_transform(train_dict)

In [134]:
X_val = dict_vectorizer.transform(val_dict)

In [135]:
Y_train = df_train['duration'].values
Y_val = df_val['duration'].values

In [136]:
lr = LinearRegression()

In [137]:
lr.fit(X_train, Y_train)
Y_pred = lr.predict(X_train)
rmse = mean_squared_error(Y_train, Y_pred, squared=False)
print(f'RMSE: {rmse}')

RMSE: 7.649261929771859


In [138]:
Y_val_pred = lr.predict(X_val)
rmse_val = mean_squared_error(Y_val, Y_val_pred, squared=False)
print(f'RMSE: {rmse_val}')

RMSE: 7.811818933419717
